# 🛒 Customer Purchase Behavior Analysis
Complete Workflow (Google Colab)


In [ ]:

# 📂 Upload CSV files into Colab
from google.colab import files

uploaded = files.upload()


In [ ]:

# 📚 Import Libraries
import pandas as pd

# Load datasets
customers = pd.read_csv("customers.csv")
products = pd.read_csv("products.csv")
orders = pd.read_csv("orders.csv")

# Preview the data
print("Customers Data:")
display(customers.head())

print("\nProducts Data:")
display(products.head())

print("\nOrders Data:")
display(orders.head())


In [ ]:

# 🧹 Data Cleaning & Preparation

# Convert dates to datetime
customers['signup_date'] = pd.to_datetime(customers['signup_date'])
orders['order_date'] = pd.to_datetime(orders['order_date'])

# Remove duplicates (if any)
customers.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
orders.drop_duplicates(inplace=True)

# Handle missing values
customers.fillna({'gender': 'Other', 'loyalty_tier': 'Unknown'}, inplace=True)
products.fillna({'stock_flag': 'In Stock'}, inplace=True)

# Create derived fields
orders['net_order_value'] = orders['order_value'] - orders['discount']

print("✅ Cleaning done")


In [ ]:

# 🔗 Merge Orders with Customers & Products for Analysis
merged = orders.merge(customers, on="customer_id", how="left") \
               .merge(products, on="product_id", how="left")

print("Merged dataset shape:", merged.shape)
display(merged.head())


In [ ]:

# 📊 RFM Analysis (Recency, Frequency, Monetary)

# Latest date in dataset
latest_date = merged['order_date'].max()

# RFM table
rfm = merged.groupby('customer_id').agg({
    'order_date': lambda x: (latest_date - x.max()).days,
    'order_id': 'count',
    'net_order_value': 'sum'
}).reset_index()

rfm.columns = ['customer_id', 'Recency', 'Frequency', 'Monetary']

print("✅ RFM Table created")
display(rfm.head())

# Save RFM data
rfm.to_csv("rfm_analysis.csv", index=False)


In [ ]:

# 💾 Save Cleaned Data for Tableau / Power BI Dashboard
merged.to_csv("cleaned_orders.csv", index=False)
print("✅ Cleaned dataset saved as cleaned_orders.csv")
print("✅ RFM analysis saved as rfm_analysis.csv")
